In [ ]:
%%writefile app.py

import streamlit as st
import pandas as pd
import numpy as np
import altair as alt

# Create a simple DataFrame for demonstration
df = pd.DataFrame({
    'x': np.random.rand(100),
    'y': np.random.rand(100),
})

# Streamlit app begins here
st.title('Streamlit Docker Example')

# Display a DataFrame
st.write('Displaying a random DataFrame:')
st.write(df)

# Create an interactive scatter plot using Altair
scatter_chart = alt.Chart(df).mark_circle().encode(
    x='x',
    y='y',
    tooltip=['x', 'y']
).interactive()

# Display the scatter plot
st.write('Interactive Scatter Plot:')
st.altair_chart(scatter_chart, use_container_width=True)



In [ ]:
%%writefile requirements.txt

streamlit==0.89.0
altair==4.1.0
pandas==1.3.3
numpy==1.21.2
protobuf==3.20.0

In [ ]:
import digitalhub as dh

proj = dh.get_or_create_project("project-container")

In [ ]:
# Create function and run
function = proj.new_function(name="streamlit",
                             kind="container",
                             base_image="python:3.9-slim",
                             command="streamlit",
                             args=["run", "app.py", "--server.fileWatcherType=none"])

In [ ]:
# Build the container
run_build = function.run(action="build",
                         code_src="app.py",
                         instructions=[
                             "apt-get update",
                             "apt-get install -y build-essential curl software-properties-common git",
                             "rm -rf var/lib/apt/lists/*",
                             "pip3 install -r requirements.txt"])

In [ ]:
# Wait until run is completed
run_build.refresh()

In [ ]:
# Serve the streamlit container
run_serve = function.run("serve",
                         service_ports= [{"port": 8085, "target_port": 8501}],
                         service_type="ClusterIP")

# To access the streamlit container, you need to espose the service with KRM and API Gateway